**Set up**

In [39]:
import sys 
print(sys.executable)
!python -V
# !conda list -n Gridmet_Tools_IS

/Users/msleckman/miniconda3/envs/gridmet_tools_IS/bin/python
Python 3.9.9


In [28]:
import pandas as pd
import numpy as np
import zipfile
import pygeos
import folium
import pickle
import time
import hvplot.pandas
import hvplot.xarray
import geopandas as gpd

In [29]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geoviews

In [1]:
import xagg as xa
import grd2shp_xagg
import xarray as xr

**Analysis**

In [33]:
start_date = '1979-01-01'
end_date = '1989-01-01'
lon_min = -79
lon_max = -67
lat_min = 36
lat_max = 42

_Reset these var for drbb aoi_

In [35]:

tmax_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmx_1979_CurrentYear_CONUS.nc'
tmin_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmn_1979_CurrentYear_CONUS.nc'
prcp_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_pr_1979_CurrentYear_CONUS.nc'
# swave_rad_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_srad_1979_CurrentYear_CONUS.nc'
# wind_vel_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_vs_1979_CurrentYear_CONUS.nc'
# rmax_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_rmax_1979_CurrentYear_CONUS.nc'
# rmin_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_rmin_1979_CurrentYear_CONUS.nc'
# sph_url = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_sph_1979_CurrentYear_CONUS.nc'

ds_prcp = xr.open_dataset(prcp_url+'#fillmismatch')
ds_tmin = xr.open_dataset(tmin_url+'#fillmismatch')
ds_tmax = xr.open_dataset(tmax_url+'#fillmismatch')

# subset gridMET data
ds_prcp_1 = ds_prcp.sel(day=slice(start_date, end_date),
                        lon=slice(lon_min, lon_max),
                        lat=slice(lat_max, lat_min))

ds_tmin_1 = ds_tmin.sel(day=slice(start_date, end_date),
                        lon=slice(lon_min, lon_max),
                        lat=slice(lat_max, lat_min))

ds_tmax_1 = ds_tmax.sel(day=slice(start_date, end_date),
                        lon=slice(lon_min, lon_max),
                        lat=slice(lat_max, lat_min))

# ds_tmax_1.load()
# ds_tmin_1.load()
# ds_prcp_1.load()

gdf_nhru02 = gpd.read_file('./data/nhru_02/nhru_02.shp')
#gdf_gdb = gpd.read_file('./data/GeospatialFabricFeatures_02.gdb', layer="nhru")
#ds_wght = xr.open_dataset('./data/tmmx_1979.nc')

In [37]:
ds_tmax_1

<xarray.Dataset>
Dimensions:                    (lat: 144, crs: 1, lon: 287, day: 3654)
Coordinates:
  * lat                        (lat) float64 41.98 41.94 41.9 ... 36.07 36.03
  * crs                        (crs) float32 3.0
  * lon                        (lon) float64 -78.97 -78.93 ... -67.1 -67.06
  * day                        (day) datetime64[ns] 1979-01-01 ... 1989-01-01
Data variables:
    daily_maximum_temperature  (day, lat, lon) float32 ...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.0
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       18 January 2022
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

**Calculate weights and save to file for reuse:**

In [38]:
# uncomment below if first time through notebook to generate weights

start = time.perf_counter()
weightmap = xa.pixel_overlaps(ds_tmax_1, gdf_nhru02)
end = time.perf_counter()
print(f'finished agg in {round(end-start, 2)} second(s)')


creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
finished agg in 71.11 second(s)


In [14]:
print(type(weightmap))
weightmap

<class 'xagg.classes.weightmap'>


In [15]:
# save weights for future use
with open('./data/nhru_02_weights.pickle', 'wb') as file:
    pickle.dump(weightmap, file)

**Initialise grd2shp_xagg:**

In [16]:
g2s = grd2shp_xagg.Grd2ShpXagg()
g2s.initialize(
    grd=[ds_tmax_1, ds_tmin_1, ds_prcp_1],
    shp=gdf_nhru02,
    wght_file='./data/nhru_02_weights.pickle',
    time_var='day',
    lat_var='lat',
    lon_var='lon',
    var=['daily_maximum_temperature', 'daily_minimum_temperature', 'precipitation_amount'],
    var_output=['tmax', 'tmin', 'prcp'],
    ctype=0,
)

numtimesteps: 7 and Start date: 1979-01-01


In [17]:
ds_tmin_1

<xarray.Dataset>
Dimensions:                    (lat: 144, crs: 1, lon: 287, day: 7)
Coordinates:
  * lat                        (lat) float64 41.98 41.94 41.9 ... 36.07 36.03
  * crs                        (crs) float32 3.0
  * lon                        (lon) float64 -78.97 -78.93 ... -67.1 -67.06
  * day                        (day) datetime64[ns] 1979-01-01 ... 1979-01-07
Data variables:
    daily_minimum_temperature  (day, lat, lon) float32 ...
Attributes: (12/19)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.0
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    date:                       18 January 2022
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

**Run the regridding:**

In [18]:
start = time.perf_counter()
g2s.run_weights()
end = time.perf_counter()
print(f'finished agg in {round(end-start, 2)} second(s)')

generating mapped vales for daily_maximum_temperature ...
adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating daily_maximum_temperature...
all variables aggregated to polygons!
finished mapped values for daily_maximum_temperature
generating mapped vales for daily_minimum_temperature ...
adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating daily_minimum_temperature...
all variables aggregated to polygons!
finished mapped values for daily_minimum_temperature
generating mapped vales for precipitation_amount ...
adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating 

**Output the results to the netcdf file:**

In [26]:
print(type(g2s))
output_prefix = "tmp_"
g2s.write_gm_file(opath=".", prefix=output_prefix)

## may into `locale` value to localize translation error

<class 'grd2shp_xagg.grd2shp_xagg.Grd2ShpXagg'>
output path exists


/Users/msleckman/miniconda3/envs/gridmet_tools_IS/lib/python3.9/site-packages/grd2shp_xagg/grd2shp_xagg.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = self.gdf.geometry.centroid


ValueError: Provide a `locale` value to localize translation.

In [133]:
type(g2s)

grd2shp_xagg.grd2shp_xagg.Grd2ShpXagg

**load and inspect the generated netCDF file**

In [21]:
xr_mapped = xr.open_dataset('{}_climate_2022_01_20.nc'.format(output_prefix, decode_times=False)

In [22]:
xr_mapped

<xarray.Dataset>
Dimensions:  (time: 7, geomid: 4827)
Coordinates:
  * time     (time) float32 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
  * geomid   (geomid) int32 0 1 2 3 4 5 6 ... 4820 4821 4822 4823 4824 4825 4826
Data variables:
    hru_lat  (geomid) float32 ...
    hru_lon  (geomid) float32 ...
    tmax     (geomid, time) float64 ...
Attributes:
    Conventions:  CF-1.8
    featureType:  timeSeries
    history:

**Create plots of original and mapped data**

In [284]:
ds_tmax_1["daily_maximum_temperature"] = ds_tmax_1["daily_maximum_temperature"] - 459.67
gdf_nhru02["tmax"] = xr_mapped["tmax"][:,0]
gdf_nhru02["tmin"] = xr_mapped["tmin"][:,0]
gdf_nhru02["prcp"] = xr_mapped["prcp"][:,0]


In [1]:
# gdf_nhru02.plot(column = 'tmax', legend = True).set_title('tmax')
# gdf_nhru02.plot(column = 'tmin', legend = True).set_title('tmin')
# gdf_nhru02.plot(column = 'tmin', legend = True).set_title('prcp')

Plot


In [ ]:

# mapped = gdf.hvplot.polygons(geo=True, c="tmax", xlim=[-74, -67], ylim=[40, 49], crs=ccrs.epsg(3857), line_width=0.01, title="Mapped") 
# gridmet = ds_tmax_1.daily_maximum_temperature.hvplot.quadmesh(x='lon', y='lat', geo=True, xlim=[-74, -67], ylim=[40, 49], crs=ccrs.epsg(3857), title="gridMET")
# gridmet + mapped
